# Demo for how GoodData Python SDK can be used inside notebooks

[GoodData Python SDK](https://www.gooddata.com/docs/cloud/api-and-sdk/python-sdk/) is an open-source set of libraries derived from GoodData OpenAPI specification.
It provides an additional layer of abstraction on top of the raw APIs.
Besides gooddata-sdk (core) library, we provide gooddata-pandas exposing report results as data frames.

## Import libraries

In [5]:
from pathlib import Path
import pandas as pd 
from gooddata_sdk import GoodDataSdk
from gooddata_pandas import GoodPandas

## Init GoodData

In [4]:
profiles= Path.home() / ".gooddata" / "profiles.yaml"
gd_sdk = GoodDataSdk.create_from_profile(profile="demo_cicd", profiles_path=profiles)
gd_pandas = GoodPandas.create_from_profile(profile="demo_cicd", profiles_path=profiles)
workspace_id = "cicd_demo_production"
gd_frames = gd_pandas.data_frames(workspace_id)

## List Metrics in a Workspace

In [7]:
metrics = gd_sdk.catalog_workspace_content.get_metrics_catalog(workspace_id)
metrics_list = [{"id": m.id, "title": m.title} for m in metrics]
df = pd.DataFrame(metrics_list)
df

,id,title
0,avg_days_to_solve,Average days to solve
1,contribution_-_count_of_merged_pull_requests,Contribution - count of merged pull requests
2,contribution_of_top_25_users_-_merged_pr,Contribution of TOP 25% users - merged PR
3,contribution_-_stargazers,Contribution - stargazers
4,count_commit_comments,Count of commit comments
5,count_commits,Count of commits
6,count_merged_pull_requests,Count of merged pull requests
7,count_nodraft_pull_requests,Count of non draft pull requests
8,count_pull_requests,Count of pull requests
9,sum_days_to_solve,Sum days to solve


## Execute Already Defined Report

In [8]:
insight_df = gd_frames.for_insight('contribution_of_top_25_users_per_repository')
insight_df

contribution_of_top_25_users_-_merged_pr
closed_at.month repo_name                                                           
2019-09-01      gooddata-ui-sdk                                             0.500000
2019-10-01      gooddata-ui-sdk                                             0.357143
2019-11-01      gooddata-ui-sdk                                             0.850000
2019-12-01      gooddata-ui-sdk                                             0.680000
2020-01-01      gooddata-ui-sdk                                             0.411765
...                                                                              ...
2024-02-01      gooddata-dashboard-plugins                                  1.000000
                gooddata-python-sdk                                         0.750000
                gooddata-ui-sdk                                             0.615385
2024-03-01      gooddata-python-sdk                                         1.000000
                gooddata-ui-sdk                                             0.750000

[121 rows x 1 columns]

## Execute a Custom Report

In [22]:
from gooddata_sdk import ExecutionDefinition, Attribute, SimpleMetric, ObjId, RelativeDateFilter
exec_def = ExecutionDefinition(
    attributes=[
        Attribute(local_id="created_at", label="created_at.month"),
        Attribute(local_id="repo_name", label="repo_name"),
    ],
    metrics=[
        SimpleMetric(local_id="count_commits", item=ObjId(id="commit_id", type="label"), aggregation="COUNT"),
        SimpleMetric(local_id="count_pull_requests", item=ObjId(id="count_pull_requests", type="metric")),
    ],
    filters=[RelativeDateFilter(dataset=ObjId(id="created_at", type="dataset"), granularity="YEAR", from_shift=-2, to_shift=0)],
    dimensions=[["created_at"], ["repo_name", "measureGroup"]],
)
df, df_metadata = gd_frames.for_exec_def(exec_def=exec_def)

# use result ID from computation above and generate dataframe just from it
df_from_result_id, df_metadata_from_result_id = gd_frames.for_exec_result_id(
    result_id=df_metadata.execution_response.result_id,
)
df_from_result_id

Repo name               gooddata-dashboard-plugins                         \
                                     count_commits Count of pull requests   
Created at - Month/Year                                                     
2022-01                                       18.0                    4.0   
2022-02                                        3.0                    1.0   
2022-03                                        NaN                    1.0   
2022-04                                       18.0                    4.0   
2022-05                                        9.0                    5.0   
2022-06                                        NaN                    2.0   
2022-07                                        NaN                    NaN   
2022-08                                        2.0                    1.0   
2022-09                                        NaN                    NaN   
2022-10                                        2.0                    3.0   
2022-11                                        5.0                    1.0   
2022-12                                        NaN                    NaN   
2023-01                                        NaN                    NaN   
2023-02                                        4.0                    3.0   
2023-03                                        8.0                    4.0   
2023-04                                        5.0                    1.0   
2023-05                                        8.0                    4.0   
2023-06                                        1.0                    1.0   
2023-07                                        NaN                    NaN   
2023-08                                        2.0                    1.0   
2023-09                                        NaN                    3.0   
2023-10                                        8.0                    1.0   
2023-11                                        NaN                    3.0   
2023-12                                       18.0                    5.0   
2024-01                                        2.0                    7.0   
2024-02                                       14.0                    7.0   
2024-03                                        NaN                    NaN   

Repo name               gooddata-public-demos                         \
                                count_commits Count of pull requests   
Created at - Month/Year                                                
2022-01                                   NaN                    NaN   
2022-02                                   NaN                    NaN   
2022-03                                   NaN                    NaN   
2022-04                                   NaN                    NaN   
2022-05                                   7.0                    8.0   
2022-06                                  11.0                   14.0   
2022-07                                   5.0                    5.0   
2022-08                                   4.0                    4.0   
2022-09                                   2.0                    2.0   
2022-10                                   6.0                    6.0   
2022-11                                   1.0                    1.0   
2022-12                                   1.0                    4.0   
2023-01                                   2.0                    1.0   
2023-02                                   1.0                    1.0   
2023-03                                   9.0                    6.0   
2023-04                                   2.0                    2.0   
2023-05                                   5.0                    4.0   
2023-06                                   NaN                    NaN   
2023-07                                   3.0                    3.0   
2023-08                                  16.0                   13.0   
2023-09                          

## Conclusion

This notebook is just a quick onboarding into how to leverage GoodData Python SDK libraries in a notebook.
I recommend you to check our [DOC](https://www.gooddata.com/docs/cloud/api-and-sdk/python-sdk/) for more details.
I tested the same approach in DeepNote and it worked.
There is no reason why it should work in any other notebook.